In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim
from torch.utils.data import DataLoader , TensorDataset
import tensorflow as tf
from   torch.optim.lr_scheduler import ExponentialLR as ExponentialLR
#from torch.optim.lr_scheduler import ExponentialLR

In [ ]:
data = pd.read_csv(r"C:\Users\Downloads\MLT\NN\Bank_Personal_Loan_Modelling.csv")

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.drop(['ID'] , inplace = True , axis = 1)

In [ ]:
x = data.drop(['Personal Loan'] , axis = 1).values
y = data['Personal Loan'].values

In [ ]:
x = torch.tensor(x , dtype = torch.float64)
y = torch.tensor(y)
y = y.type(torch.LongTensor)

In [ ]:
y

In [ ]:
class NN(nn.Module):
    def __init__(self , input_size , hidden1 , hidden2 ,  output_size):
        super().__init__()
        self.input_ = nn.Linear(input_size , hidden1)
        self.hidden1_ = nn.Linear(hidden1 , hidden2)
        self.out = nn.Linear(hidden2 , output_size)
    def forward(self , x):
        x = f.relu(self.input_(x))
        x = torch.sigmoid(self.hidden1_(x))
        x = self.out(x)
        x = f.sigmoid(x)
        return x

In [ ]:
x.shape

In [ ]:
model = NN(12 , 10 , 8 , 2)

In [ ]:
#Parameters
model.parameters

In [ ]:
loss_function = nn.CrossEntropyLoss()

In [ ]:
loss_function

In [ ]:
optimizer = torch.optim.Adam( model.parameters(), lr=100)
scheduler = ExponentialLR(optimizer, gamma=0.9)

In [ ]:
n_epochs = 1000

In [ ]:
data = TensorDataset(x , y)
data = DataLoader(data )

In [ ]:
with tf.device('/gpu:0'):
    final_losses = []
    for i in range(n_epochs):
        y_pred = model(x.float())
        loss = loss_function(y_pred , y)
        final_losses.append(loss.item())
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        if (i%10 == 0):
            print("Epoch {} has loss of {}".format(i , loss.item()))

In [ ]:
plt.plot(range(1 , n_epochs+1) , final_losses)

In [ ]:
predictions = []
with torch.no_grad():
    for i , data in enumerate(x):
        y_pred = (model(data.float()))
        predictions.append(y_pred.argmax().item())

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y , predictions))

#GA

In [ ]:
import pandas as pd
import numpy as np

In [ ]:

#intializing the population
import random
populations = ([[random.randint(0,1) for  i in range(6)] for j in range(4)])
print(populations)

best = -100000
parents=[]
new_population =[]

#fitness score

def fitness_score():
    global populations,best
    fit_value=[]
    fit_score =[]
    chromosome_value = 0
    for i in range(4):
        for j in range(5,0,-1):
            chromosome_value += populations[i][j](2*(5-j))
            if populations[i][0] ==1:
                chromosome_value = -1*chromosome_value
            else:
                chromosome_value
        print(chromosome_value)   
        fit_value.append(-(chromosome_value**2)+5)
    
    fit_value,populations = zip(*sorted(zip(fit_value,populations),reverse= True))
    print(fit_value)
    print(populations)
    best = fit_value[0]
    



def select_parents():
    global parents
    parents = populations[0:2]
    print(parents)



def crossover():
    global parents
    cross_point = random.randint(0,5)
    parents = parents + tuple([(parents[0][0:cross_point+1]+parents[1][cross_point+1:6])])
    parents = parents + tuple([(parents[1][0:cross_point+1]+parents[0][cross_point+1:6])])
    print(parents)

   


def mutation():
    global populations,parents
    mute = random.randint(0, 29)
    if mute == 10:
        x= random.randint(0, 4)
        y = random.randint(0,5)
        parents[x][y] = 1 - parents[x][y]
    populations = parents
    print(populations)




for i in range(100):
    fitness_score()
    select_parents()
    crossover() 
    mutation()
print("the best score is",best) 
print("Sequence is",populations[0])

#AN

In [ ]:
import numpy as np

NV = 10

def formPheremoneMatrix(graph):
    
    nv = len(graph)
    pheremones = np.ones((nv,nv))
        
    return pheremones
           
def chooseNextVertex(graph, pheremones, currPos):
    graph = 1/graph
    denominator = np.dot(graph[currPos], pheremones[currPos])
    numerator = graph[currPos] * pheremones[currPos]
    
    probabilities = numerator/denominator
    
    
    rouletteWheel = np.cumsum(probabilities)
    
    rouletteBall = np.random.random()
    
    nextVertex = np.where(rouletteWheel >= rouletteBall)[0][0]
    
    return nextVertex

def traverse(graph, pheremones, start, end):
    curr = start
    path = [curr]
    cost = 0
    prev = start
    
    while curr != end:
        nextVertex = chooseNextVertex(graph, pheremones, curr)
        
        while nextVertex == prev:
            nextVertex = chooseNextVertex(graph, pheremones, curr)
        
        cost += graph[curr][nextVertex]
        path += [nextVertex]
        prev = curr
        curr = nextVertex
        
    
    return path, cost
    
    

def releaseGeneration(graph, pheremones, start, end, size = 10):
    paths = []
    costs = []
    for i in range(size):
        p, c = traverse(graph, pheremones, start, end)
        paths += [p]
        costs += [c]
    costs = np.array(costs)    
    
    return paths, costs
    
def updatePheremones(paths, costs, pheremones, decay = 0):
    pheremones = (1-decay)*pheremones
    
    costs = 1/costs
    for p in range(len(paths)):
        path = paths[p]
        for i in range(len(path) - 1):
            pheremones[path[i]][path[i+1]] += costs[p]
    
    return pheremones
            

def generateProblem(size, density):
    graph = np.full((size, size), np.inf)
    
    for i in range(len(graph)):
        for j in range(i, len(graph)):
            if np.random.random() < density:
                if i!=j:
                    w = np.random.randint(1,20)
                    graph[i][j] = w
                    graph[j][i] = w
                    
    return graph
#%%
graph = generateProblem(NV, 0.5)

#%%
ph = formPheremoneMatrix(graph)
print(graph)

#%%
gen = 0
SIZE = 100

for i in range(100):
    p,c = releaseGeneration(graph, ph, 0, 7, SIZE)
    ph = updatePheremones(p, c, ph, decay = 0)
    gen += 1

    print("--",gen,"--")   
    c = np.array(c)
    unique, counts = np.unique(c, return_counts=True)
    print(unique)
    print(counts)
    
    if len(np.where(counts > SIZE//2)[0]) == 1:
        break

#ca

In [ ]:
import sklearn
import pandas as pd
import pygad
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from pygad.kerasga import KerasGA
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
df = df.drop('Unnamed: 0',axis = 1)

In [ ]:
X = df.drop('target',axis = 1)
y = df['target']

In [ ]:
len(X.columns)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, 
                                                    test_size=0.33, 
                                                    random_state=42,
                                                    shuffle = True)

In [ ]:
y_train=np_utils.to_categorical(y_train,num_classes=2)
y_test=np_utils.to_categorical(y_test,num_classes=2)

In [ ]:
model = Sequential([tf.keras.layers.Dense(12,input_shape = (12,),activation = 'relu'),
                    tf.keras.layers.Dense(8,activation = 'relu'),
                    tf.keras.layers.Dense(16, activation = 'relu'),
                    tf.keras.layers.Dense(8,activation = 'relu'),
                    tf.keras.layers.Dense(2,activation = 'softmax')
                   ])

In [ ]:
model.compile(optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
def fitness_func(solution, sol_idx):
    global X_train, y_train, keras_ga, model
    
    model_weights_matrix = pygad.kerasga.model_weights_as_matrix(model = model,weights_vector = solution)
    
    model.set_weights(weights = model_weights_matrix)
    
    predictions = model.predict(X_train)
    
    loss = tf.keras.losses.CategoricalCrossentropy()
    
    solution_fitness = 1.0/(loss(y_train,predictions).numpy() + 0.0000000001)
    
    return solution_fitness

In [ ]:
def acceptance_function(population, fitness):
    belief_space = population[fitness.index(max(fitness))]
    return belief_space

In [ ]:
def influence_function(belief_space,population):
    sim = cosine_similarity(belief_space.reshape(1,-1),population)
    return sim

In [ ]:
def parent_selection(sim,population):
    # Get the indices of the two highest values using argsort
    highest_indices = np.argsort(sim[0])[-2:]

    # Get the values at the indices using fancy indexing
    highest_values = sim[0][highest_indices]
    parent_1 = population[highest_indices[0]]
    parent_2 = population[highest_indices[1]]
    return parent_1,parent_2

In [ ]:
def crossover(parent1, parent2):
    """
    Performs a crossover operation between two parents, producing two new children.

    Args:
        parent1 (numpy.ndarray): The first parent.
        parent2 (numpy.ndarray): The second parent.

    Returns:
        A tuple of two children produced by crossover.
    """
    # Determine the length of the parents and the crossover point.
    length = parent1.shape[0]
    crossover_point = np.random.randint(1, length - 1)

    # Create the first child by combining the parents.
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))

    # Create the second child by combining the parents in reverse order.
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))

    return child1, child2

In [ ]:
def mutation(array):
    """
    Adds a randomly generated value to 15% of the elements in a numpy array.

    Args:
        array (numpy.ndarray): The input array.

    Returns:
        A numpy array with mutations applied.
    """
    # Determine the number of elements to mutate.
    num_mutations = int(0.15 * array.size)

    # Generate random indices for the elements to mutate.
    mutation_indices = np.random.choice(array.size, size=num_mutations, replace=False)

    # Add a random value to the selected elements.
    array[mutation_indices] += np.random.randn(num_mutations)

    return array

In [ ]:
def prune_population(population,child1, child2):
    population = np.vstack((population, child1))
    population = np.vstack((population,child2))
# Calculate the fitness of each solution in the population.
    fitness_values = np.zeros((population.shape[0],))
    for i in range(population.shape[0]):
        fitness_values[i] = fitness_func(population[i], i)

    # Sort the population by fitness.
    sorted_indices = np.argsort(fitness_values)
    sorted_population = population[sorted_indices]

    # Return the top 10 elements.
    return sorted_population[-10:]

In [ ]:
def cultural_algorithm(population):
    
    fitness = []
    for idx, solution in enumerate(population):
        fitness.append(fitness_func(solution,idx))
    
    belief_space = acceptance_function(population,fitness)
    similarity_score = influence_function(belief_space,population)
    parent1,parent2 = parent_selection(sim,population)
    child1,child2 = crossover(parent1, parent2)
    child1 = mutation(child1)
    child2 = mutation(child2)
    population = prune_population(population,child1, child2)
    return population

In [ ]:
def set_final_weights(population):
    fitness = []
    for idx, solution in enumerate(population):
        fitness.append(fitness_func(solution,idx))
    model_weights_matrix = pygad.kerasga.model_weights_as_matrix(model = model,weights_vector = population[fitness.index(max(fitness))])
    
    model.set_weights(weights = model_weights_matrix)

In [ ]:
# Get the shapes of the model's weights
weights_shapes = [w.shape for w in model.get_weights()]

# Compute the total number of weights
num_weights = np.sum([np.prod(s) for s in weights_shapes])


In [ ]:
population = np.random.uniform(size = (10,num_weights))

In [ ]:
for i in range(3):
    population = cultural_algorithm(population)

In [ ]:
model.evaluate(X_test,y_test)